# 🔍 Smart Lens — YOLOv8 Threat Detection Model Training

**Complete training pipeline for the Smart Lens CCTV Surveillance System**

---

### What this notebook does:
1. ✅ Connects Google Drive (saves everything permanently)
2. ✅ Clones your GitHub repo
3. ✅ Downloads Roboflow labeled dataset (750 images)
4. ✅ Explores & validates the dataset
5. ✅ Trains YOLOv8 with optimized settings for small datasets
6. ✅ Evaluates model performance (mAP, confusion matrix, PR curves)
7. ✅ Tests on sample images/video
8. ✅ Exports best model for deployment
9. ✅ Saves everything to Drive + pushes to GitHub

### ⚡ Before running:
- Go to **Runtime → Change runtime type → T4 GPU → Save**

---

### 📊 Training Strategy for 750 Images

| Setting | Value | Why |
|---------|-------|-----|
| **Model** | YOLOv8s (small) | Best accuracy/speed balance for small datasets |
| **Epochs** | 200 | Small dataset needs more epochs to converge |
| **Early Stopping** | patience=50 | Auto-stops if no improvement for 50 epochs |
| **Image Size** | 640 | Standard for YOLO, good for CCTV frames |
| **Batch Size** | 16 | Optimal for T4 GPU (16GB VRAM) |
| **Augmentation** | Heavy | Critical for small datasets — prevents overfitting |
| **Optimizer** | AdamW | Better convergence than SGD for small data |
| **Learning Rate** | 0.001 → cosine decay | Smooth convergence |
| **Pretrained** | COCO weights | Transfer learning is essential with 750 images |

---
## 📦 Step 0: Setup Environment

In [ ]:
#@title 0.1 — Mount Google Drive (permanent storage)
from google.colab import drive
drive.mount('/content/drive')

# Create project folder on Drive
import os
DRIVE_PROJECT = '/content/drive/MyDrive/Smart-Lens-FYP'
DRIVE_MODELS  = f'{DRIVE_PROJECT}/trained_models'
DRIVE_RESULTS = f'{DRIVE_PROJECT}/training_results'
DRIVE_DATASET = f'{DRIVE_PROJECT}/dataset'

for d in [DRIVE_PROJECT, DRIVE_MODELS, DRIVE_RESULTS, DRIVE_DATASET]:
    os.makedirs(d, exist_ok=True)

print('✅ Google Drive mounted!')
print(f'📂 Project folder: {DRIVE_PROJECT}')

In [ ]:
#@title 0.2 — Verify GPU
import torch

if not torch.cuda.is_available():
    raise RuntimeError('❌ No GPU! Go to Runtime → Change runtime type → T4 GPU')

gpu_name = torch.cuda.get_device_name(0)
gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
print(f'✅ GPU: {gpu_name} ({gpu_mem:.1f} GB VRAM)')
print(f'   PyTorch: {torch.__version__}')
print(f'   CUDA: {torch.version.cuda}')

In [ ]:
#@title 0.3 — Install dependencies
!pip install -q ultralytics roboflow

import ultralytics
ultralytics.checks()
print('✅ All dependencies installed!')

In [ ]:
#@title 0.4 — Clone GitHub repo
import os

REPO_URL = 'https://github.com/Alee-Razaa/Smart-Lens-FYP.git'
REPO_DIR = '/content/Smart-Lens-FYP'

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

# Configure git
!git config --global user.name 'Ali Raza Memon'
!git config --global user.email 'alirazamemon.bsaif22@iba-suk.edu.pk'

os.chdir(REPO_DIR)
print(f'✅ Repo ready at: {os.getcwd()}')

---
## 📥 Step 1: Download & Explore Dataset

In [ ]:
#@title 1.1 — Download dataset from Roboflow
from roboflow import Roboflow
import shutil

# Check if dataset already exists on Drive (skip re-download)
DATASET_DIR = '/content/dataset'

if os.path.exists(f'{DRIVE_DATASET}/data.yaml'):
    print('📦 Dataset found on Drive! Copying to runtime (faster)...')
    if os.path.exists(DATASET_DIR):
        shutil.rmtree(DATASET_DIR)
    shutil.copytree(DRIVE_DATASET, DATASET_DIR)
    print(f'✅ Dataset loaded from Drive')
else:
    print('📥 Downloading dataset from Roboflow...')
    rf = Roboflow(api_key='7QsEv54uizzlrvPZ972Z')
    project = rf.workspace('fpy').project('smart-survellaince-lens-2')
    version = project.version(1)
    dataset = version.download('yolov8', location=DATASET_DIR)

    # Backup to Drive
    print('💾 Backing up dataset to Google Drive...')
    if os.path.exists(DRIVE_DATASET):
        shutil.rmtree(DRIVE_DATASET)
    shutil.copytree(DATASET_DIR, DRIVE_DATASET)
    print(f'✅ Dataset downloaded & backed up to Drive')

# Show structure
!find {DATASET_DIR} -type d | head -20
print(f'\n📄 data.yaml contents:')
!cat {DATASET_DIR}/data.yaml

In [ ]:
#@title 1.2 — Dataset statistics & analysis
import yaml
import glob
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

# Load data.yaml
with open(f'{DATASET_DIR}/data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

class_names = data_config['names']
num_classes = len(class_names) if isinstance(class_names, list) else len(class_names.values())

# Handle both list and dict formats for class names
if isinstance(class_names, dict):
    class_list = [class_names[i] for i in sorted(class_names.keys())]
else:
    class_list = class_names

print(f'📊 DATASET SUMMARY')
print(f'===================')
print(f'Classes ({num_classes}): {class_list}')

# Count images per split
splits = {}
for split in ['train', 'valid', 'test']:
    img_path = f'{DATASET_DIR}/{split}/images'
    if os.path.exists(img_path):
        count = len(glob.glob(f'{img_path}/*'))
        splits[split] = count
        print(f'  {split}: {count} images')

total = sum(splits.values())
print(f'  TOTAL: {total} images')

# Count labels per class
all_labels = []
for split in ['train', 'valid', 'test']:
    label_path = f'{DATASET_DIR}/{split}/labels'
    if os.path.exists(label_path):
        for txt_file in glob.glob(f'{label_path}/*.txt'):
            with open(txt_file, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if parts:
                        all_labels.append(int(parts[0]))

label_counts = Counter(all_labels)
print(f'\n📊 LABEL DISTRIBUTION')
print(f'=====================')
for cls_id in sorted(label_counts.keys()):
    name = class_list[cls_id] if cls_id < len(class_list) else f'class_{cls_id}'
    count = label_counts[cls_id]
    bar = '█' * (count // 5)
    print(f'  [{cls_id}] {name:20s}: {count:5d} {bar}')

print(f'  TOTAL annotations: {len(all_labels)}')

# Plot distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Split distribution
axes[0].bar(splits.keys(), splits.values(), color=['#2ecc71', '#3498db', '#e74c3c'])
axes[0].set_title('Images per Split', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Count')
for i, (k, v) in enumerate(splits.items()):
    axes[0].text(i, v + 5, str(v), ha='center', fontweight='bold')

# Class distribution
cls_names_sorted = [class_list[i] if i < len(class_list) else f'cls_{i}' for i in sorted(label_counts.keys())]
cls_counts_sorted = [label_counts[i] for i in sorted(label_counts.keys())]
colors = plt.cm.Set2(np.linspace(0, 1, len(cls_names_sorted)))
axes[1].barh(cls_names_sorted, cls_counts_sorted, color=colors)
axes[1].set_title('Annotations per Class', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Count')
for i, v in enumerate(cls_counts_sorted):
    axes[1].text(v + 2, i, str(v), va='center', fontweight='bold')

plt.tight_layout()
plt.savefig(f'{DRIVE_RESULTS}/dataset_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('\n💾 Saved: dataset_analysis.png')

In [ ]:
#@title 1.3 — Visualize sample images with labels
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np

# Get random training images
train_images = glob.glob(f'{DATASET_DIR}/train/images/*')
samples = random.sample(train_images, min(9, len(train_images)))

# Color map for classes
COLORS = [(0,255,0), (0,0,255), (255,0,0), (255,255,0), (255,0,255),
          (0,255,255), (128,0,255), (255,128,0)]

fig, axes = plt.subplots(3, 3, figsize=(18, 18))
axes = axes.flatten()

for idx, img_path in enumerate(samples):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # Read corresponding label
    label_path = img_path.replace('/images/', '/labels/')
    label_path = os.path.splitext(label_path)[0] + '.txt'

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls_id = int(parts[0])
                    xc, yc, bw, bh = [float(x) for x in parts[1:5]]

                    # Convert YOLO format to pixel coordinates
                    x1 = int((xc - bw/2) * w)
                    y1 = int((yc - bh/2) * h)
                    x2 = int((xc + bw/2) * w)
                    y2 = int((yc + bh/2) * h)

                    color = COLORS[cls_id % len(COLORS)]
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                    label = class_list[cls_id] if cls_id < len(class_list) else f'cls_{cls_id}'
                    cv2.putText(img, label, (x1, y1-8), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    axes[idx].imshow(img)
    axes[idx].set_title(os.path.basename(img_path), fontsize=9)
    axes[idx].axis('off')

plt.suptitle('📸 Sample Training Images with Annotations', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{DRIVE_RESULTS}/sample_images.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 🔧 Step 2: Fix data.yaml paths for Colab

In [ ]:
#@title 2.1 — Update data.yaml with correct Colab paths
import yaml

with open(f'{DATASET_DIR}/data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

# Fix paths for Colab
data_config['train'] = f'{DATASET_DIR}/train/images'
data_config['val']   = f'{DATASET_DIR}/valid/images'

# Add test path if exists
if os.path.exists(f'{DATASET_DIR}/test/images'):
    data_config['test'] = f'{DATASET_DIR}/test/images'

# Save updated config
DATA_YAML = f'{DATASET_DIR}/data.yaml'
with open(DATA_YAML, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print('✅ data.yaml updated:')
!cat {DATA_YAML}

---
## 🚀 Step 3: Train YOLOv8 Model

### Training Strategy for 750 Images

With only **750 images**, we must:
1. **Use pretrained weights** (COCO) — transfer learning is critical
2. **Heavy augmentation** — artificially increase dataset diversity
3. **More epochs** (200) — small datasets need more passes to learn
4. **Early stopping** (patience=50) — auto-stop if overfitting
5. **YOLOv8s** (small) — large models overfit on small data

| Epochs Guide for Dataset Size |
|------|
| < 500 images → 250-300 epochs |
| 500-1000 images → 150-200 epochs |
| 1000-5000 images → 100-150 epochs |
| 5000+ images → 50-100 epochs |

In [ ]:
#@title 3.1 — Configure training parameters

# ============================================================
#  TRAINING CONFIGURATION — Optimized for 750 images
# ============================================================

CONFIG = {
    # Model
    'model': 'yolov8s.pt',          # Small model — best for <1000 images
                                     # Options: yolov8n.pt (nano/fastest),
                                     #          yolov8s.pt (small/balanced) ✅
                                     #          yolov8m.pt (medium/more accurate but risk overfit)

    # Training
    'epochs': 200,                   # 200 epochs for 750 images
    'patience': 50,                  # Early stopping — stops if no improvement for 50 epochs
    'batch': 16,                     # Batch size — optimal for T4 16GB VRAM
    'imgsz': 640,                    # Image size — standard for YOLO
    'device': 0,                     # GPU device

    # Optimizer
    'optimizer': 'AdamW',            # Better than SGD for small datasets
    'lr0': 0.001,                    # Initial learning rate
    'lrf': 0.01,                     # Final LR = lr0 * lrf (cosine decay)
    'weight_decay': 0.0005,          # L2 regularization
    'warmup_epochs': 5,              # Warmup for stable start

    # Augmentation (HEAVY for small dataset)
    'hsv_h': 0.015,                  # Hue augmentation
    'hsv_s': 0.7,                    # Saturation augmentation
    'hsv_v': 0.4,                    # Value/brightness augmentation
    'degrees': 10.0,                 # Rotation ±10°
    'translate': 0.2,                # Translation ±20%
    'scale': 0.5,                    # Scale ±50%
    'shear': 5.0,                    # Shear ±5°
    'flipud': 0.0,                   # No vertical flip (surveillance is upright)
    'fliplr': 0.5,                   # Horizontal flip 50%
    'mosaic': 1.0,                   # Mosaic augmentation (combine 4 images)
    'mixup': 0.15,                   # MixUp 15% (blend 2 images)
    'copy_paste': 0.1,               # Copy-Paste augmentation 10%
    'erasing': 0.4,                  # Random Erasing 40% (simulates occlusion)

    # Regularization
    'dropout': 0.1,                  # Light dropout to prevent overfitting
    'close_mosaic': 20,              # Disable mosaic for last 20 epochs (fine-tune)

    # Saving
    'project': '/content/runs',
    'name': 'smart_lens_v1',
    'save': True,
    'save_period': 25,               # Save checkpoint every 25 epochs
    'plots': True,                   # Generate training plots
    'exist_ok': True,
}

print('✅ Training Configuration:')
print(f'   Model:      {CONFIG["model"]}')
print(f'   Epochs:     {CONFIG["epochs"]} (early stop patience={CONFIG["patience"]})')
print(f'   Batch Size: {CONFIG["batch"]}')
print(f'   Image Size: {CONFIG["imgsz"]}')
print(f'   Optimizer:  {CONFIG["optimizer"]} (lr={CONFIG["lr0"]})')
print(f'   Augmentation: HEAVY (mosaic={CONFIG["mosaic"]}, mixup={CONFIG["mixup"]}, erasing={CONFIG["erasing"]})')
print(f'\n⏱️ Estimated training time: ~45-90 minutes on T4 GPU')

In [ ]:
#@title 3.2 — 🚀 START TRAINING (run this and wait)
from ultralytics import YOLO
import time

print('🚀 Starting YOLOv8 training...')
print(f'   Dataset: {DATA_YAML}')
print(f'   This will take ~45-90 minutes on T4 GPU')
print('=' * 60)

start_time = time.time()

# Load pretrained model
model = YOLO(CONFIG['model'])

# Train
results = model.train(
    data=DATA_YAML,
    epochs=CONFIG['epochs'],
    patience=CONFIG['patience'],
    batch=CONFIG['batch'],
    imgsz=CONFIG['imgsz'],
    device=CONFIG['device'],
    optimizer=CONFIG['optimizer'],
    lr0=CONFIG['lr0'],
    lrf=CONFIG['lrf'],
    weight_decay=CONFIG['weight_decay'],
    warmup_epochs=CONFIG['warmup_epochs'],
    hsv_h=CONFIG['hsv_h'],
    hsv_s=CONFIG['hsv_s'],
    hsv_v=CONFIG['hsv_v'],
    degrees=CONFIG['degrees'],
    translate=CONFIG['translate'],
    scale=CONFIG['scale'],
    shear=CONFIG['shear'],
    flipud=CONFIG['flipud'],
    fliplr=CONFIG['fliplr'],
    mosaic=CONFIG['mosaic'],
    mixup=CONFIG['mixup'],
    copy_paste=CONFIG['copy_paste'],
    erasing=CONFIG['erasing'],
    dropout=CONFIG['dropout'],
    close_mosaic=CONFIG['close_mosaic'],
    project=CONFIG['project'],
    name=CONFIG['name'],
    save=CONFIG['save'],
    save_period=CONFIG['save_period'],
    plots=CONFIG['plots'],
    exist_ok=CONFIG['exist_ok'],
    verbose=True,
)

elapsed = time.time() - start_time
print('\n' + '=' * 60)
print(f'✅ Training complete! Time: {elapsed/60:.1f} minutes')
print(f'📂 Results saved to: {CONFIG["project"]}/{CONFIG["name"]}')

---
## 📊 Step 4: Evaluate Model Performance

In [ ]:
#@title 4.1 — Show training curves
from IPython.display import Image, display

RESULTS_DIR = f'{CONFIG["project"]}/{CONFIG["name"]}'

# Training curves
print('📈 TRAINING CURVES')
print('=' * 40)
if os.path.exists(f'{RESULTS_DIR}/results.png'):
    display(Image(filename=f'{RESULTS_DIR}/results.png', width=900))
else:
    print('⚠️ results.png not found')

In [ ]:
#@title 4.2 — Confusion Matrix
print('🔢 CONFUSION MATRIX')
print('=' * 40)
for fname in ['confusion_matrix.png', 'confusion_matrix_normalized.png']:
    fpath = f'{RESULTS_DIR}/{fname}'
    if os.path.exists(fpath):
        print(f'\n{fname}:')
        display(Image(filename=fpath, width=700))

In [ ]:
#@title 4.3 — Precision-Recall & F1 Curves
print('📊 PRECISION-RECALL & F1 CURVES')
print('=' * 40)
for fname in ['PR_curve.png', 'P_curve.png', 'R_curve.png', 'F1_curve.png']:
    fpath = f'{RESULTS_DIR}/{fname}'
    if os.path.exists(fpath):
        print(f'\n{fname}:')
        display(Image(filename=fpath, width=700))

In [ ]:
#@title 4.4 — Validate on validation set (detailed metrics)
from ultralytics import YOLO

# Load best weights
best_model = YOLO(f'{RESULTS_DIR}/weights/best.pt')

# Run validation
val_results = best_model.val(
    data=DATA_YAML,
    imgsz=640,
    batch=16,
    device=0,
    plots=True,
    verbose=True
)

print('\n' + '=' * 60)
print('📊 VALIDATION RESULTS')
print('=' * 60)
print(f'  mAP50:      {val_results.box.map50:.4f}')
print(f'  mAP50-95:   {val_results.box.map:.4f}')
print(f'  Precision:  {val_results.box.mp:.4f}')
print(f'  Recall:     {val_results.box.mr:.4f}')
print('\n  Per-class AP50:')
for i, ap in enumerate(val_results.box.ap50):
    name = class_list[i] if i < len(class_list) else f'class_{i}'
    bar = '█' * int(ap * 30)
    print(f'    [{i}] {name:20s}: {ap:.4f} {bar}')

# Quality assessment
map50 = val_results.box.map50
print('\n' + '=' * 60)
if map50 >= 0.7:
    print(f'✅ GOOD! mAP50={map50:.2%} — Model is performing well')
elif map50 >= 0.5:
    print(f'⚠️ DECENT. mAP50={map50:.2%} — Consider more data or fine-tuning')
else:
    print(f'❌ NEEDS IMPROVEMENT. mAP50={map50:.2%} — See recommendations below')
    print('   → Add more labeled images (target 1500+)')
    print('   → Check label quality in Roboflow')
    print('   → Try yolov8m.pt or increase epochs')

---
## 🧪 Step 5: Test Model on Sample Images

In [ ]:
#@title 5.1 — Run inference on validation images
import glob
import random

# Get validation images
val_images = glob.glob(f'{DATASET_DIR}/valid/images/*')
test_images = glob.glob(f'{DATASET_DIR}/test/images/*') if os.path.exists(f'{DATASET_DIR}/test/images') else []
all_test = val_images + test_images

# Pick random samples
samples = random.sample(all_test, min(12, len(all_test)))

# Run inference
results = best_model.predict(
    source=samples,
    conf=0.4,
    iou=0.5,
    save=True,
    project='/content/runs/predict',
    name='test_samples',
    exist_ok=True
)

# Display results
fig, axes = plt.subplots(3, 4, figsize=(24, 16))
axes = axes.flatten()

pred_dir = '/content/runs/predict/test_samples'
pred_images = sorted(glob.glob(f'{pred_dir}/*'))[:12]

for idx, img_path in enumerate(pred_images):
    if idx >= 12:
        break
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[idx].imshow(img)
    axes[idx].axis('off')

# Hide unused axes
for idx in range(len(pred_images), 12):
    axes[idx].axis('off')

plt.suptitle('🔍 Model Predictions on Test Images', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{DRIVE_RESULTS}/test_predictions.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'\n✅ Inference complete! {len(pred_images)} images processed')

In [ ]:
#@title 5.2 — Speed benchmark (FPS test)
import time
import numpy as np

# Warm up GPU
dummy = np.random.randint(0, 255, (640, 640, 3), dtype=np.uint8)
for _ in range(5):
    best_model.predict(dummy, verbose=False)

# Benchmark
times = []
for _ in range(50):
    start = time.time()
    best_model.predict(dummy, verbose=False)
    times.append(time.time() - start)

avg_ms = np.mean(times) * 1000
fps = 1000 / avg_ms

print(f'⚡ SPEED BENCHMARK (T4 GPU)')
print(f'==========================')
print(f'  Average inference: {avg_ms:.1f} ms per frame')
print(f'  FPS: {fps:.1f} frames/second')
print(f'  Min: {np.min(times)*1000:.1f} ms | Max: {np.max(times)*1000:.1f} ms')

if fps >= 30:
    print(f'\n✅ Real-time capable! ({fps:.0f} FPS > 30 FPS requirement)')
elif fps >= 15:
    print(f'\n⚠️ Near real-time ({fps:.0f} FPS) — acceptable for surveillance')
else:
    print(f'\n❌ Below real-time ({fps:.0f} FPS) — consider yolov8n.pt for speed')

---
## 📦 Step 6: Export Model for Deployment

In [ ]:
#@title 6.1 — Export best model (PyTorch + ONNX)
import shutil

BEST_PT = f'{RESULTS_DIR}/weights/best.pt'
LAST_PT = f'{RESULTS_DIR}/weights/last.pt'

# Export to ONNX (for production deployment)
print('📦 Exporting model to ONNX...')
best_model.export(format='onnx', imgsz=640, simplify=True)
BEST_ONNX = BEST_PT.replace('.pt', '.onnx')

print(f'\n✅ Models exported:')
print(f'  PyTorch:  {BEST_PT} ({os.path.getsize(BEST_PT)/1e6:.1f} MB)')
if os.path.exists(BEST_ONNX):
    print(f'  ONNX:     {BEST_ONNX} ({os.path.getsize(BEST_ONNX)/1e6:.1f} MB)')

---
## 💾 Step 7: Save Everything to Drive + Push to GitHub

In [ ]:
#@title 7.1 — Save trained model + results to Google Drive
import shutil
import datetime

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
model_name = f'smart_lens_v1_{timestamp}'

# Save best model to Drive
drive_model_path = f'{DRIVE_MODELS}/{model_name}'
os.makedirs(drive_model_path, exist_ok=True)

# Copy weights
shutil.copy2(BEST_PT, f'{drive_model_path}/best.pt')
shutil.copy2(LAST_PT, f'{drive_model_path}/last.pt')
if os.path.exists(BEST_ONNX):
    shutil.copy2(BEST_ONNX, f'{drive_model_path}/best.onnx')

# Copy all training results/plots
for fname in os.listdir(RESULTS_DIR):
    fpath = f'{RESULTS_DIR}/{fname}'
    if os.path.isfile(fpath):
        shutil.copy2(fpath, f'{DRIVE_RESULTS}/{fname}')

# Save training config
import json
with open(f'{drive_model_path}/training_config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

# Save validation metrics
metrics = {
    'mAP50': float(val_results.box.map50),
    'mAP50_95': float(val_results.box.map),
    'precision': float(val_results.box.mp),
    'recall': float(val_results.box.mr),
    'training_time_min': round(elapsed / 60, 1),
    'fps': round(fps, 1),
    'timestamp': timestamp,
    'classes': class_list if isinstance(class_list, list) else list(class_list),
}
with open(f'{drive_model_path}/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f'\n✅ SAVED TO GOOGLE DRIVE')
print(f'========================')
print(f'  📂 Model:   {drive_model_path}/')
print(f'  📂 Results: {DRIVE_RESULTS}/')
print(f'\n  Files saved:')
for f in os.listdir(drive_model_path):
    size = os.path.getsize(f'{drive_model_path}/{f}') / 1e6
    print(f'    📄 {f} ({size:.1f} MB)')

In [ ]:
#@title 7.2 — Push training results to GitHub
import json

# Save a lightweight results summary to the repo (no large model files)
os.chdir(REPO_DIR)

# Create results directory in repo
os.makedirs(f'{REPO_DIR}/results', exist_ok=True)

# Save metrics
with open(f'{REPO_DIR}/results/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

# Copy key plots (small files only)
for fname in ['results.png', 'confusion_matrix.png', 'PR_curve.png', 'F1_curve.png']:
    src = f'{RESULTS_DIR}/{fname}'
    if os.path.exists(src):
        shutil.copy2(src, f'{REPO_DIR}/results/{fname}')

# Commit and push
!cd {REPO_DIR} && git add -A
!cd {REPO_DIR} && git status
!cd {REPO_DIR} && git commit -m "Add YOLOv8 training results — mAP50={metrics['mAP50']:.4f}"
!cd {REPO_DIR} && git push origin master

print(f'\n✅ Results pushed to GitHub!')
print(f'   mAP50: {metrics["mAP50"]:.4f}')
print(f'   Repo: https://github.com/Alee-Razaa/Smart-Lens-FYP')

---
## 📋 Step 8: Training Summary & Next Steps

In [ ]:
#@title 8.1 — Print final summary

print('=' * 70)
print('🔍 SMART LENS — TRAINING SUMMARY')
print('=' * 70)
print(f'''
  📊 Dataset:        {total} images, {num_classes} classes
  🤖 Model:          YOLOv8s (pretrained COCO → fine-tuned)
  ⏱️  Training Time:  {elapsed/60:.1f} minutes
  
  📈 METRICS:
     mAP50:          {metrics["mAP50"]:.4f} ({metrics["mAP50"]:.1%})
     mAP50-95:       {metrics["mAP50_95"]:.4f}
     Precision:      {metrics["precision"]:.4f}
     Recall:         {metrics["recall"]:.4f}
     FPS:            {metrics["fps"]} frames/sec
  
  💾 SAVED TO:
     Google Drive:   {drive_model_path}/
     GitHub:         https://github.com/Alee-Razaa/Smart-Lens-FYP
  
  🔑 TO LOAD THIS MODEL LATER:
     from ultralytics import YOLO
     model = YOLO('{drive_model_path}/best.pt')
''')

# Recommendations based on results
print('  📌 NEXT STEPS:')
if metrics['mAP50'] < 0.5:
    print('     ❌ mAP is low. Recommendations:')
    print('        1. Add more labeled images (target 1500+) on Roboflow')
    print('        2. Review label quality — remove bad annotations')
    print('        3. Balance classes — under-represented classes need more samples')
    print('        4. Try yolov8m.pt with lower learning rate')
elif metrics['mAP50'] < 0.7:
    print('     ⚠️ mAP is decent. To improve:')
    print('        1. Add 300-500 more images per weak class')
    print('        2. Fine-tune: lower lr to 0.0005, train 100 more epochs from best.pt')
    print('        3. Try Test-Time Augmentation (TTA) for better inference')
else:
    print('     ✅ mAP is good! Ready for integration.')
    print('        1. Export to ONNX/TensorRT for faster inference')
    print('        2. Build the FastAPI backend to serve the model')
    print('        3. Connect to RTSP cameras for live detection')

print('\n' + '=' * 70)

---
## 🔄 Bonus: Resume Training / Fine-Tune

If you want to **continue training** from the saved model (e.g., after adding more data):

In [ ]:
#@title [OPTIONAL] Resume training from last checkpoint
# Uncomment and run if you want to resume/fine-tune

# from ultralytics import YOLO

# # Option A: Resume from where training stopped
# model = YOLO(f'{DRIVE_MODELS}/{model_name}/last.pt')
# model.train(resume=True)

# # Option B: Fine-tune best model on new/updated data
# model = YOLO(f'{DRIVE_MODELS}/{model_name}/best.pt')
# model.train(
#     data=DATA_YAML,
#     epochs=100,
#     lr0=0.0005,       # Lower LR for fine-tuning
#     patience=30,
#     freeze=10,        # Freeze first 10 layers (keep learned features)
#     project='/content/runs',
#     name='smart_lens_v2_finetune',
# )